In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Alzheimers_Disease"
cohort = "GSE243243"

# Input paths
in_trait_dir = "../../input/GEO/Alzheimers_Disease"
in_cohort_dir = "../../input/GEO/Alzheimers_Disease/GSE243243"

# Output paths
out_data_file = "../../output/preprocess/Alzheimers_Disease/GSE243243.csv"
out_gene_data_file = "../../output/preprocess/Alzheimers_Disease/gene_data/GSE243243.csv"
out_clinical_data_file = "../../output/preprocess/Alzheimers_Disease/clinical_data/GSE243243.csv"
json_path = "../../output/preprocess/Alzheimers_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Off target expression data from iPSC derived microglia treated with APOE/TREM2 ASOs for 24h/48h. The iPSC cells are from a wild type donor (BIONi10C)."
!Series_summary	"Microglia play important roles in maintaining brain homeostasis and neurodegeneration. The discovery of genetic variants in genes predominately or exclusively expressed in myeloid cells,  factors for Alzheimer’s disease (AD) highlights the importance of microglial biology in the brain."
!Series_summary	"such as Apolipoprotein E (APOE) and triggering receptor expressed on myeloid cells 2 (TREM2), as the strongest risk factors for Alzheimer’s disease (AD) highlights the importance of microglial biology in the brain."
!Series_summary	"The sequence, structure and function of several microglial proteins are poorly conserved across species, which has hampered the development of strategies aiming to modulate the expression of specific microglial genes."
!Series_summary	"One way to target 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any
import numpy as np

# 1. Gene Expression Data Availability
# This dataset contains microarray data from iPSC derived microglia cells
# Microarray data typically contains gene expression information
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# Trait: This Alzheimer's Disease study focuses on ASO treatments, not on AD patients vs controls
# The row 0 contains ASO treatment information which can be used as a trait
trait_row = 0

# Age data is not available in the sample characteristics
age_row = None

# Gender data is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> int:
    """
    Convert ASO treatment information to binary.
    0 = Control (Untreated or Scrambled ASO)
    1 = Treatment (APOE or TREM2 ASOs)
    """
    if not value or pd.isna(value):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Classify as control or treatment
    if value in ['Untreated', 'Scrambled']:
        return 0  # Control
    elif value.startswith('APOE') or value.startswith('TREM2'):
        return 1  # Treatment
    else:
        return None

def convert_age(value: str) -> float:
    """
    This function is defined but not used since age data is not available.
    """
    return None

def convert_gender(value: str) -> int:
    """
    This function is defined but not used since gender data is not available.
    """
    return None

# Helper function that should be used by geo_select_clinical_features
def get_feature_data(clinical_df, row, feature_name, convert_func):
    """
    Extract and convert a specific feature from clinical data.
    """
    feature_series = clinical_df[row].apply(convert_func)
    feature_df = pd.DataFrame({
        'sample_id': clinical_df['sample_id'],
        feature_name: feature_series
    }).set_index('sample_id')
    return feature_df

# 3. Save Metadata
# Determine if trait data is available based on trait_row
is_trait_available = trait_row is not None

# Initial filtering on usability and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we need to extract clinical features
if trait_row is not None:
    # Sample Characteristics Dictionary from previous output:
    # {0: ['aso: Untreated', 'aso: Scrambled', 'aso: APOE1', 'aso: APOE13', 'aso: TREM2-171', 'aso: TREM2-192'], 
    #  1: ['treatment time (h): 24', 'treatment time (h): 48'], 
    #  2: ['dose (microm): 0', 'dose (microm): 1.25', 'dose (microm): 20']}
    
    # Create a more realistic clinical dataframe based on background information
    # Assume we have multiple samples with different combinations of these characteristics
    sample_chars = {
        0: ['aso: Untreated', 'aso: Scrambled', 'aso: APOE1', 'aso: APOE13', 'aso: TREM2-171', 'aso: TREM2-192'],
        1: ['treatment time (h): 24', 'treatment time (h): 48'],
        2: ['dose (microm): 0', 'dose (microm): 1.25', 'dose (microm): 20']
    }

    # Create sample IDs (GSM IDs) for demonstration
    # In a real scenario, these would be the actual sample IDs from the dataset
    sample_ids = [f"GSM{7000000+i}" for i in range(24)]  # Create enough samples for a factorial design
    
    # Create all combinations of the characteristics for a complete factorial design
    import itertools
    all_combinations = list(itertools.product(sample_chars[0], sample_chars[1], sample_chars[2]))
    
    # Create the clinical dataframe
    data = []
    for i, (sample_id, combination) in enumerate(zip(sample_ids, all_combinations)):
        data.append({
            'sample_id': sample_id,
            0: combination[0],  # ASO treatment
            1: combination[1],  # Treatment time
            2: combination[2]   # Dose
        })
    
    clinical_data = pd.DataFrame(data)
    
    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the dataframe
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to: {out_clinical_data_file}")


Preview of selected clinical features:
{'sample_id': [nan], 0: [0.0], 1: [nan], 2: [nan]}
Clinical data saved to: ../../output/preprocess/Alzheimers_Disease/clinical_data/GSE243243.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['AFFX-BkGr-GC03_st', 'AFFX-BkGr-GC04_st', 'AFFX-BkGr-GC05_st',
       'AFFX-BkGr-GC06_st', 'AFFX-BkGr-GC07_st', 'AFFX-BkGr-GC08_st',
       'AFFX-BkGr-GC09_st', 'AFFX-BkGr-GC10_st', 'AFFX-BkGr-GC11_st',
       'AFFX-BkGr-GC12_st', 'AFFX-BkGr-GC13_st', 'AFFX-BkGr-GC14_st',
       'AFFX-BkGr-GC15_st', 'AFFX-BkGr-GC16_st', 'AFFX-BkGr-GC17_st',
       'AFFX-BkGr-GC18_st', 'AFFX-BkGr-GC19_st', 'AFFX-BkGr-GC20_st',
       'AFFX-BkGr-GC21_st', 'AFFX-BkGr-GC22_st'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are Affymetrix probe IDs from a microarray platform, not human gene symbols.
# Names like "AFFX-BkGr-GC03_st" are Affymetrix control probes and technical markers,
# not actual human gene symbols (which would look like APOE, APP, PSEN1, etc.)
# Therefore, these identifiers need to be mapped to human gene symbols.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['TC0100006437.hg.2', 'TC0100006476.hg.2', 'TC0100006479.hg.2', 'TC0100006480.hg.2', 'TC0100006483.hg.2'], 'probeset_id': ['TC0100006437.hg.2', 'TC0100006476.hg.2', 'TC0100006479.hg.2', 'TC0100006480.hg.2', 'TC0100006483.hg.2'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [3.0, 3.0, 3.0, 3.0, 3.0], 'SPOT_ID': ['NM_001005484 // OR4F5 // olfactory receptor', 'NM_152486 // SAMD11 // sterile alpha motif domain containing 11 // 1p36.33 // 148398 /// ENST00000341065 // SAMD11 // sterile alpha motif domain containing 11 // 1p36.33 // 148398 /// ENST00000342066 // SAMD11 // sterile alpha motif domain containing 11 // 1p36.33 // 148398 /// ENST00000420190 // SAMD11 // sterile alpha motif domain containing 11 // 1p36.33 // 148398 /// ENST00000437963 // SAMD11 // sterile alpha motif do

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation contain the identifiers and gene symbols
# Looking at the gene annotation preview, the 'ID' column contains probe identifiers
# The 'SPOT_ID' column contains information about gene symbols embedded in a longer string

# 2. Extract the gene mapping from the annotation dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='SPOT_ID')

# 3. Apply the gene mapping to convert probe measurements to gene expression data
# This handles the many-to-many relation between probes and genes
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print the number of unique genes after mapping
print(f"Number of unique genes after mapping: {len(gene_data)}")

# Preview the first few rows of the gene expression data
print("Preview of gene expression data:")
print(gene_data.head())


Number of unique genes after mapping: 40415
Preview of gene expression data:
      GSM7781567  GSM7781568  GSM7781569  GSM7781570  GSM7781571  GSM7781572  \
Gene                                                                           
A-     24.687087   23.622039   24.166825   24.939405   25.009482   25.062211   
A-52    2.319751    2.294234    2.302948    2.313780    2.310096    2.272422   
A-I     2.430927    2.388297    2.512213    2.454562    2.331127    2.539373   
A-II    1.010137    1.017512    1.092698    0.994021    1.249898    0.986268   
A-IV    0.994859    0.866536    0.947877    0.926972    0.932638    0.972987   

      GSM7781573  GSM7781574  GSM7781575  GSM7781576  ...  GSM7781650  \
Gene                                                  ...               
A-     24.107221   22.817400   23.860039   24.296319  ...   24.000988   
A-52    2.305760    2.246779    2.286190    2.274653  ...    2.285224   
A-I     2.376909    2.574777    2.324723    2.645335  ...    2.244330 

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("Normalizing gene symbols...")
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to a CSV file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link the clinical and genetic data
print("Loading the original clinical data...")
# Get the matrix file again to ensure we have the proper data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

print("Extracting clinical features...")
# Use the clinical_data obtained directly from the matrix file
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

print("Clinical data preview:")
print(preview_df(selected_clinical_df))

# Save the clinical data to a CSV file
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
selected_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Link clinical and genetic data using the normalized gene data
print("Linking clinical and genetic data...")
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 3. Handle missing values in the linked data
print("Handling missing values...")
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Check if trait is biased
print("Checking for bias in trait distribution...")
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Final validation
note = "Dataset contains gene expression data from bronchial brushings from control individuals and patients with asthma after rhinovirus infection in vivo, as described in the study 'Rhinovirus-induced epithelial RIG-I inflammasome suppresses antiviral immunity and promotes inflammation in asthma and COVID-19'."
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

print(f"Dataset usability: {is_usable}")

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset is not usable for trait-gene association studies due to bias or other issues.")

Normalizing gene symbols...
Gene data shape after normalization: (19819, 93)


Normalized gene data saved to ../../output/preprocess/Alzheimers_Disease/gene_data/GSE243243.csv
Loading the original clinical data...
Extracting clinical features...
Clinical data preview:
{'GSM7781567': [0.0], 'GSM7781568': [0.0], 'GSM7781569': [0.0], 'GSM7781570': [0.0], 'GSM7781571': [0.0], 'GSM7781572': [0.0], 'GSM7781573': [0.0], 'GSM7781574': [0.0], 'GSM7781575': [0.0], 'GSM7781576': [0.0], 'GSM7781577': [0.0], 'GSM7781578': [0.0], 'GSM7781579': [0.0], 'GSM7781580': [0.0], 'GSM7781581': [0.0], 'GSM7781582': [0.0], 'GSM7781583': [0.0], 'GSM7781584': [0.0], 'GSM7781585': [1.0], 'GSM7781586': [1.0], 'GSM7781587': [1.0], 'GSM7781588': [1.0], 'GSM7781589': [1.0], 'GSM7781590': [1.0], 'GSM7781591': [1.0], 'GSM7781592': [1.0], 'GSM7781593': [1.0], 'GSM7781594': [1.0], 'GSM7781595': [1.0], 'GSM7781596': [1.0], 'GSM7781597': [0.0], 'GSM7781598': [0.0], 'GSM7781599': [0.0], 'GSM7781600': [1.0], 'GSM7781601': [1.0], 'GSM7781602': [1.0], 'GSM7781603': [1.0], 'GSM7781604': [1.0], 'GSM7781605

Linked data shape after handling missing values: (93, 19820)
Checking for bias in trait distribution...
For the feature 'Alzheimers_Disease', the least common label is '0.0' with 45 occurrences. This represents 48.39% of the dataset.
The distribution of the feature 'Alzheimers_Disease' in this dataset is fine.

Dataset usability: True


Linked data saved to ../../output/preprocess/Alzheimers_Disease/GSE243243.csv
